# Improving Performance: Comparing Different Models

In our `training.ipynb` notebook, we built a Decision Tree model that achieved 93% accuracy. In this notebook, we will build on that work by testing several other powerful models to see if we can improve upon that result. We will use the exact same training and testing data to ensure a fair comparison.

### Setup and Data Loading

First, we repeat the same initial steps: import libraries, load the data, and split it into the same training and testing sets.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Import the models we want to test
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Load and prepare the data
df = pd.read_csv('../data/iris.csv')
X = df.drop('target', axis=1)
y = df['target']

# Use the same random_state to get the exact same split as before
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Creating a Dictionary of Models

To make our comparison clean and easy, we will store our models in a dictionary. This allows us to loop through them and run the same evaluation steps for each one.

In [4]:
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Support Vector Machine': SVC(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

### Train and Evaluate Each Model

Now, we will iterate through our dictionary. For each model, we will:
1. Train it on the training data.
2. Make predictions on the test data.
3. Print a classification report to see its performance in detail.

In [5]:
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Print the results
    print(f'--- {name} ---')
    print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
    print(classification_report(y_test, y_pred))
    print('' + '-' * 40)

--- Decision Tree ---
Accuracy: 0.93
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30

----------------------------------------
--- Random Forest ---
Accuracy: 0.90
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.82      0.90      0.86        10
           2       0.89      0.80      0.84        10

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30

----------------------------------------
--- Support Vector Machine ---
Accuracy: 0.97
              precision    recall  f

### 4. Feature Importance

The Random Forest model can provide us with "feature importances," which tell us which features the model found most useful for making its predictions. Let's visualize this.

In [ ]:
# Get the feature importances from the trained Random Forest model
importances = models['Random Forest'].feature_importances_
feature_names = X.columns

# Create a DataFrame for visualization
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# Plot the feature importances
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance_df)
plt.title('Feature Importance from Random Forest')
plt.show()

**Observation:** As expected, the model confirms that `petal width` and `petal length` are by far the most important features for classifying the flowers. This aligns perfectly with our findings from the EDA.

In [6]:
import joblib

model = models['K-Nearest Neighbors']  # Choose the KNN model for saving

# Define the filename for the model
model_filename = '../models/iris_K-Nearest_Neighbors.joblib'

# Save the model to the file
joblib.dump(model, model_filename)

['../models/iris_K-Nearest_Neighbors.joblib']

### 5. Conclusion: Analyzing the Results


#### K-Nearest Neighbors (KNN): 100% Accuracy
- **How it Works:** KNN is a simple but powerful model. It classifies a new flower by looking at the 'k' closest flowers in the training data and taking a majority vote. For the Iris dataset, the species are so well-clustered that a new flower is almost always surrounded by neighbors of its own kind. 
- **Why it Performed Best:** It achieved a perfect score because the feature space of the Iris dataset is very well-defined. The clusters of species are dense and clearly separated, making it easy for KNN to find the correct neighbors and make the right prediction every time.

#### Support Vector Machine (SVM): 97% Accuracy
- **How it Works:** An SVM works by finding the optimal hyperplane or boundary that best separates the classes. It tries to maximize the margin (the distance) between the different classes.
- **Why it Performed Well:** SVMs are excellent at finding the subtle, non-linear boundaries that might exist between classes. It achieved 97% accuracy, only making one mistake between species 1 and 2. This shows it was very effective at defining the decision boundaries, even for the two more similar species.

#### Decision Tree: 93% Accuracy
- **How it Works:** A Decision Tree makes predictions by creating a set of if-then-else rules based on the features. It splits the data at each node to create the purest possible child nodes.
- **Why it Performed Well (but not perfectly):** Our original model still performed very well. Its 93% accuracy shows that simple, interpretable rules are enough to solve most of this problem. Its errors occurred where the boundary between species 1 and 2 is a bit fuzzy, which can sometimes challenge a single tree.

#### Random Forest: 90% Accuracy
- **How it Works:** A Random Forest is an ensemble of many Decision Trees. It builds multiple trees on different subsets of the data and features, and the final prediction is a majority vote from all the trees.
- **Why it Underperformed (in this specific case):** It is surprising that the Random Forest performed worse than the single Decision Tree. While usually more robust, a Random Forest introduces randomness in its feature selection for each tree. For a simple dataset like Iris where only a couple of features are critical (like petal width and length), this randomness might have occasionally caused some trees to be built without access to the most important features, leading to a few incorrect votes in the final ensemble. This is a rare case and a good learning example that more complex models are not *always* better on simpler datasets.